In [1]:
from unidecode import unidecode
from nltk.tokenize import TweetTokenizer
#from stop_words import get_stop_words
import nltk
from string import punctuation 
from nltk.corpus import stopwords
tknzr = TweetTokenizer()

stop_words = [unidecode(stopW) for stopW in stopwords.words('spanish')]
non_words = list(punctuation)
non_words.extend(['¿', '¡', 'q', 'd', 'x', 'xq', '...', '..'])
stop_words = stop_words + non_words

In [1]:
from writereadfile import *
import word2vec
from em_utilities_LORE3 import *

In [3]:
model300 = word2vec.load('tweetsNOStemmed_words2vec300.bin')
vocabulary=list(model300.vocab)

In [4]:
politicians=['CynthiaViteri6',
'LassoGuillermo',
'pesanteztwof',
'daloes10',
'IvanEspinelM',
'Lenin',
'PacoMoncayo',
'ZuquilandaDuque',
'a_alcivar','andrespaezec','JorgeGlas','MauricioPozoEC','MonseBustamant','ramiroaguilart',
'CristinaReyesec','fcorderoc','HenryKronfle','irinacabezas','marcelaguinaga','mmcuesta',
'MoisesTacle','PaolaVintimilla','RamiroGonzalezJ','lcparodi', 
'AlbertoAcostaE','AbAlvaroNoboa', 'jaimenebotsaadi',
'LourdesTiban', 'PaulCarrascoC', 'MashiRafael', 'natasha_rojas',
'jaimenebotsaadi','MauricioRodasEC','PoliticoRadical','VeroGallardoEC','mckronfle','AndresCarrion2',
'lenhurtado',
'martharoldos',
'Presidencia_Ec']

In [2]:
tweets_candidates=load_json('all_tweets_politicans')

# Classify the tweets of the users in the file 1 with EM, consider 7 clusters and print the results 

In [17]:
users_tweets=load_json('citizensPlus_politiciansTest/users_tweetsChosen1')

In [18]:
len(users_tweets)

100

In [22]:
documents_vectors=load_json('citizensPlus_politiciansTest/tweetsChosen_vecs1')
print(len(documents_vectors))

#Initializing cluster means
data_centroids300=load_json('model300/data_centroids300')
means=data_centroids300['centroids']['7']

clusters_for_tweets=assign_clusters(documents_vectors, means)  #take the vectors that are not normalized
num_clusters=7
num_docs = np.array(documents_vectors).shape[0]
documents_vectors = normalize(documents_vectors)

63104


In [24]:
#Initializing cluster weights
weights = []
for i in range(num_clusters):
    # Compute the number of data points assigned to cluster i:
    num_assigned = np.count_nonzero(clusters_for_tweets == i)# YOUR CODE HERE
    w = float(num_assigned) / num_docs
    weights.append(w)

###Initializing covariances
covs = []
for i in range(num_clusters):
    member_rows = documents_vectors[clusters_for_tweets==i]
    cov = ((((member_rows*member_rows)-2*(member_rows.dot(np.diag(means[i])))).sum(axis=0))/member_rows.shape[0])+(np.array(means[i])*np.array(means[i]))
    cov[cov < 1e-8] = 1e-8
    covs.append(cov)
    
classification = EM_for_high_dimension(documents_vectors, means, covs, weights, cov_smoothing=1e-10)
## classification = {'weights':weights,'means':mu,'covs':Sigma,'loglik':ll_trace,'resp':resp}

63104 300 7
1000 300 7


In [32]:
print(classification['loglik'])  ##7 clusters
print(classification['weights'])

[43762815.269406162, 44904932.692161798, 44761719.986305594]
[ 0.22515379  0.04414745  0.15733301  0.08516643  0.02673618  0.12732086
  0.33414227]


In [33]:
save_json('citizensPlus_politiciansTest/responsibility_7Clus_1', classification['resp'].tolist())

In [ ]:
num_tweetsChosenPol

In [29]:
len(classification['resp'])

63104

In [36]:
politicians_classification=classification['resp'][-12122:]
pol=users_tweets[-10:]
index=0
multiProfile_users=[]
for u in pol:
    responsibilities=[]
    for t in u['tweets_chosen']:
        responsibilities.append(politicians_classification[index])
        index+=1
        #print(t[1])
        #print(np.argmax(classification['resp'][index]))
    #print(len(responsibilities))
    sum_responsibilities=(np.array(responsibilities).sum(axis=0))
    #print(sum_responsibilities)
    #print(type(sum_responsibilities))
    multidim_profile=np.around(sum_responsibilities/sum_responsibilities.sum(), decimals=3)*100
    print(u['user'],multidim_profile)
    #multiProfile_users.append((u,multidim_profile))

CynthiaViteri6 [  1.6   3.   37.2   0.7   2.5  26.5  28.5]
LassoGuillermo [  0.7  21.6  45.3   0.3   1.1  25.9   5.2]
pesanteztwof [  0.4   8.8  47.4   1.6   0.6  35.4   5.8]
daloes10 [  5.9   7.5  57.5   0.1   0.7  15.   13.3]
IvanEspinelM [  3.2   8.2  49.8   0.    2.3  29.8   6.8]
Lenin [  0.    3.3  42.5   0.    0.8  34.8  18.5]
PacoMoncayo [  1.    9.5  30.9   4.3   0.9  46.5   7.1]
ZuquilandaDuque [  0.6   4.6  43.2   0.    1.1  42.7   7.9]
a_alcivar [  0.8   1.   25.2   0.9   0.8  55.4  15.9]
andrespaezec [  1.6   3.9  43.7   5.2   1.1  21.8  22.7]


# RUN THE TEST FOR 22 CLUSTERS IN THE 4 FILES

In [5]:
num_tweetsChosenPol=[12122, 8916, 18918, 18577] ##the 399 users'tweets were divided in 4 files (90 normal users + 10 politicians)
multiProfile_users=[]
for j in range(0,2):
    print(j+1)
    users_tweets=load_json('citizensPlus_politiciansTest/users_tweetsChosen'+str(j+1))
    pol=users_tweets[-10:]
    documents_vectors=load_json('citizensPlus_politiciansTest/tweetsChosen_vecs'+str(j+1))
    
    data_centroids300=load_json('model300/data_centroids300') #load the centroids for 300dim: 
    #['3', '5', '6', '8', '10', '20', '30', '80', '7']
    
    for key in ['22']:  #data_centroids300['centroids']:
        #Initializing cluster means
        means=data_centroids300['centroids'][key]
        clusters_for_tweets=assign_clusters(documents_vectors, means)  #take the vectors that are not normalized
        num_clusters=len(means)
        print(num_clusters)
        num_docs = np.array(documents_vectors).shape[0]
        documents_vectors = normalize(documents_vectors)
        
        #Initializing cluster weights
        weights = []
        for i in range(num_clusters):
            # Compute the number of data points assigned to cluster i:
            num_assigned = np.count_nonzero(clusters_for_tweets == i)# YOUR CODE HERE
            w = float(num_assigned) / num_docs
            weights.append(w)

        ###Initializing covariances
        covs = []
        for i in range(num_clusters):
            member_rows = documents_vectors[clusters_for_tweets==i]
            cov = ((((member_rows*member_rows)-2*(member_rows.dot(np.diag(means[i])))).sum(axis=0))/member_rows.shape[0])+(np.array(means[i])*np.array(means[i]))
            cov[cov < 1e-8] = 1e-8
            covs.append(cov)
    
        classification = EM_for_high_dimension(documents_vectors, means, covs, weights, cov_smoothing=1e-10)
        ## classification = {'weights':weights,'means':mu,'covs':Sigma,'loglik':ll_trace,'resp':resp}
        
        ## RESULTS
        print(classification['loglik'])  ##7 clusters
        print(classification['weights'])
        
        save_json('citizensPlus_politiciansTest/responsibility_'+str(num_clusters)+'Clus_'+str(j+1), classification['resp'].tolist())
    
        politicians_classification=classification['resp'][-num_tweetsChosenPol[j]:]
        index=0
        for u in pol:
            responsibilities=[]
            for t in u['tweets_chosen']:
                responsibilities.append(politicians_classification[index])
                index+=1
                #print(t[1])
                #print(np.argmax(classification['resp'][index]))
            #print(len(responsibilities))
            sum_responsibilities=(np.array(responsibilities).sum(axis=0))
            #print(sum_responsibilities)
            #print(type(sum_responsibilities))
            multidim_profile=np.around(sum_responsibilities/sum_responsibilities.sum(), decimals=3)*100
            print(u['user'],multidim_profile)
            multiProfile_users.append((u,multidim_profile))
    
#save_json('citizensPlus_politiciansTest/multiprofile_politicians_ALLCLusters',multiProfile_users)

1
22
63104 300 22
1000 300 7
[46055400.632713318, 46963690.302444138, 46876439.554090865]
[ 0.01272824  0.12381111  0.02575967  0.04459186  0.00326946  0.03684904
  0.00086216  0.05262751  0.02472171  0.02163325  0.02668815  0.00327493
  0.05492082  0.02107256  0.0740434   0.03381363  0.02049235  0.01324648
  0.0845549   0.01686915  0.24614491  0.05802471]
CynthiaViteri6 [  0.1   9.6   0.5   7.2   0.1   0.1   0.   26.9   1.5   0.1   3.5   0.
   0.6   0.8  14.2   5.    0.3   1.3   3.3  21.5   3.    0.4]
LassoGuillermo [  0.1   6.7   1.2   3.5   0.    0.1   0.    6.8   1.3   0.1  21.9   0.
   0.4   2.9  36.7  12.4   0.2   3.2   1.1   0.8   0.5   0. ]
pesanteztwof [  0.4   7.9   0.9   3.6   0.    0.    0.    8.3   1.8   0.4  10.8   0.
   0.    8.   38.6  12.2   0.1   3.9   0.4   2.5   0.    0. ]
daloes10 [  0.1  12.9   1.9   3.    0.    0.3   0.   11.5   0.3   0.4   7.2   0.
   2.    2.4  33.5   6.3   1.    6.8   2.1   0.1   8.2   0. ]
IvanEspinelM [  0.   10.8   3.    2.2   0.    0.    0

TypeError: Object of type 'ndarray' is not JSON serializable

In [6]:
num_tweetsChosenPol=[12122, 8916, 18918, 18577]
multiProfile_users=[]
for j in range(2,4):
    print(j+1)
    users_tweets=load_json('citizensPlus_politiciansTest/users_tweetsChosen'+str(j+1))
    pol=users_tweets[-10:]
    documents_vectors=load_json('citizensPlus_politiciansTest/tweetsChosen_vecs'+str(j+1))
    
    data_centroids300=load_json('model300/data_centroids300') #load the centroids for 300dim: 
    #['3', '5', '6', '8', '10', '20', '30', '80', '7']
    
    for key in ['22']:  #data_centroids300['centroids']:
        #Initializing cluster means
        means=data_centroids300['centroids'][key]
        clusters_for_tweets=assign_clusters(documents_vectors, means)  #take the vectors that are not normalized
        num_clusters=len(means)
        print(num_clusters)
        num_docs = np.array(documents_vectors).shape[0]
        documents_vectors = normalize(documents_vectors)
        
        #Initializing cluster weights
        weights = []
        for i in range(num_clusters):
            # Compute the number of data points assigned to cluster i:
            num_assigned = np.count_nonzero(clusters_for_tweets == i)# YOUR CODE HERE
            w = float(num_assigned) / num_docs
            weights.append(w)

        ###Initializing covariances
        covs = []
        for i in range(num_clusters):
            member_rows = documents_vectors[clusters_for_tweets==i]
            cov = ((((member_rows*member_rows)-2*(member_rows.dot(np.diag(means[i])))).sum(axis=0))/member_rows.shape[0])+(np.array(means[i])*np.array(means[i]))
            cov[cov < 1e-8] = 1e-8
            covs.append(cov)
    
        classification = EM_for_high_dimension(documents_vectors, means, covs, weights, cov_smoothing=1e-10)
        ## classification = {'weights':weights,'means':mu,'covs':Sigma,'loglik':ll_trace,'resp':resp}
        
        ## RESULTS
        print(classification['loglik'])  ##7 clusters
        print(classification['weights'])
        
        save_json('citizensPlus_politiciansTest/responsibility_'+str(num_clusters)+'Clus_'+str(j+1), classification['resp'].tolist())
    
        politicians_classification=classification['resp'][-num_tweetsChosenPol[j]:]
        index=0
        for u in pol:
            responsibilities=[]
            for t in u['tweets_chosen']:
                responsibilities.append(politicians_classification[index])
                index+=1
                #print(t[1])
                #print(np.argmax(classification['resp'][index]))
            #print(len(responsibilities))
            sum_responsibilities=(np.array(responsibilities).sum(axis=0))
            #print(sum_responsibilities)
            #print(type(sum_responsibilities))
            multidim_profile=np.around(sum_responsibilities/sum_responsibilities.sum(), decimals=3)*100
            print(u['user'],multidim_profile)
            multiProfile_users.append((u,multidim_profile))
    
#save_json('citizensPlus_politiciansTest/multiprofile_politicians_ALLCLusters',multiProfile_users)

3
22
74240 300 22
1000 300 7
[54478903.389378667, 55837671.127430856, 55767384.27355621]
[ 0.01575009  0.08397949  0.0374589   0.04648973  0.00128564  0.03537669
  0.00283306  0.04124661  0.06734851  0.02181424  0.02185609  0.00257104
  0.03723221  0.03071455  0.08447492  0.0297835   0.0177362   0.00968001
  0.08641371  0.01331411  0.22528558  0.08735511]
MoisesTacle [  0.    2.3   2.2   4.2   0.    0.    0.    3.5   5.7   0.   18.9   0.
   0.    1.5  22.9  35.8   0.3   0.4   0.2   2.1   0.1   0. ]
PaolaVintimilla [  0.8   5.3   9.7  15.    0.1   0.5   0.   14.6   7.2   0.    1.4   0.
   1.6   1.2  11.5   3.1   0.7  14.5   5.8   1.8   5.1   0.3]
RamiroGonzalezJ [  0.    8.5   1.6   5.4   0.    0.1   0.    4.3   5.5   0.1  12.9   0.
   0.6   5.1  38.   12.1   0.5   3.2   0.9   0.5   0.7   0. ]
lcparodi [  1.5   2.5   0.5   3.6   0.    0.2   0.    1.4   0.7   0.5   0.2   0.
   0.3   0.9   1.5   0.1   0.3   0.1  10.2   0.    2.5  72.8]
AlbertoAcostaE [  0.    6.    0.   27.5   0.    0.   

TypeError: Object of type 'ndarray' is not JSON serializable